## Import Libaries

In [3]:
from natsort import natsorted, ns
# import modules to handle files
import os
import sys

# import modules for compution vision
import numpy as np
import cv2

import gdown
import shutil 

## Download U2Net, MODNet and models

In [4]:
# if repository not already in directory, clone it
if not os.path.exists('U-2-Net'):
  !git clone https://github.com/xuebinqin/U-2-Net.git

# if repository not already in directory, clone it
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet.git

# create the folder to save model
if not os.path.exists(r'./U-2-Net/saved_models/u2net/'):
    os.mkdir(r'./U-2-Net/saved_models/u2net/')

# download model 
if not os.path.isfile(r'./U-2-Net/saved_models/u2net/u2net.pth'):
    url = 'https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ'
    output = r'./U-2-Net/saved_models/u2net/u2net.pth'
    gdown.download(url, output, quiet=False)

In [5]:
# Computing TRIMAP
def compute_trimap_from_alpha(name, src_dir, dst_dir, open_size=10,
                              alpha_margin=10):
    src_path = os.path.join(src_dir, name)
    # if not os.path.exists(src_path):
    #     logger.error('"%s" dose not exist', src_path)
    #     return
    print(src_path)

    dst_path = os.path.join(dst_dir, name)
    print(dst_path)
    # if os.path.exists(dst_path):
    #     logger.info('"%s" exists, Skip...', dst_path)
    #     return

    # logger.info('Trimap for "%s"', src_path)
    alpha = cv2.imread(src_path, 0)
    print(alpha)
    assert alpha.ndim == 2

    # Compute each region
    fore = ((255 - alpha_margin) < alpha)
    back = (alpha < alpha_margin)
    unknown = ~(fore + back)
    unknown = cv2.dilate(
        unknown.astype(np.uint8),
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_size, open_size))
    ).astype(np.bool)

    # Draw
    trimap = np.zeros_like(alpha)
    trimap[fore] = 255
    trimap[unknown] = 127

    cv2.imwrite(dst_path, trimap)

In [6]:
# Loading data in correct format for MODNet training  
def dataloader(file_location):
    path = file_location
    image_path = os.listdir(path + 'images')
    gt_path = os.listdir(path + 'ground_truths')
    trimap = os.listdir(path + 'trimaps')

    # short in an order
    image_path = natsorted(image_path, alg=ns.IGNORECASE)
    gt_path = natsorted(gt_path, alg=ns.IGNORECASE)
    trimap = natsorted(trimap, alg=ns.IGNORECASE)

    return zip(image_path, trimap, gt_path)

In [45]:
# getting trimap and manging storage and memory
def get_trimap(path, image):

    # main image dir should have 3 important folders images, ground_truths, trimaps
    # (one image at a time) copy target image to ./U-2-Net/test_image
    shutil.copy2(f'{path}images/{image}', './U-2-Net/test_data/test_images')

    # generate alpha_matting
    %cd ./U-2-Net/
    !python u2net_test.py

    # generate trimap 
    %cd ..
    compute_trimap_from_alpha(f"{image[:-4]}.png", "./U-2-Net/test_data/u2net_results/", f'{path}trimaps/')

    # delete every image file from test_image and u2net_results
    os.remove(f'./U-2-Net/test_data/test_images/{image}')
    
    shutil.move(f'./U-2-Net/test_data/u2net_results/{image[:-4]}.png', f'{path}alpha_matting/')


In [47]:
path = './dataset/training_set/'
image = "ILSVRC2012_test_00000004.jpg"

get_trimap(path, image)

c:\01 D\ML project\Remove_Image_Background\U-2-Net
['c:\\01 D\\ML project\\Remove_Image_Background\\U-2-Net\\test_data\\test_images\\ILSVRC2012_test_00000004.jpg']
...load U2NET---173.6 MB
inferencing: ILSVRC2012_test_00000004.jpg
c:\01 D\ML project\Remove_Image_Background
./U-2-Net/test_data/u2net_results/ILSVRC2012_test_00000004.png
./dataset/training_set/trimaps/ILSVRC2012_test_00000004.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
C:\Users\sijal\anaconda3\envs\vision\lib\site-packages\torch\nn\functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
C:\Users\sijal\anaconda3\envs\vision\lib\site-packages\torch\nn\functional.py:3454: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_c

In [46]:
# geting trimap of every image from the dir
path = './dataset/training_set/'
images = os.listdir(f'{path}/images')

for image in images:
    get_trimap(path, image)

c:\01 D\ML project\Remove_Image_Background\U-2-Net
^C
c:\01 D\ML project\Remove_Image_Background
./U-2-Net/test_data/u2net_results/ILSVRC2012_test_00000004.png
./dataset/training_set/trimaps/ILSVRC2012_test_00000004.png
None


AttributeError: 'NoneType' object has no attribute 'ndim'

In [72]:
# generate alpha_matting
path = './dataset/training_set/'

!python get_apha_matting.py path

Traceback (most recent call last):
  File "get_apha_matting.py", line 16, in <module>
    from data_loader import RescaleT
ModuleNotFoundError: No module named 'data_loader'


In [69]:
%cd ..

c:\01 D\ML project\Remove_Image_Background


In [70]:
os.getcwd()

'c:\\01 D\\ML project\\Remove_Image_Background'